In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster("local").setAppName("category-review-average")
sc = SparkContext(conf=conf)

22/07/25 23:46:52 WARN Utils: Your hostname, singyeongdeog-ui-Macmini.local resolves to a loopback address: 127.0.0.1; using 222.98.22.103 instead (on interface en0)
22/07/25 23:46:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/25 23:46:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
directory = "/Users/singyeongdeog/Documents/github_code/TIL/til-by-topic/big-data/1-spark/data"
filename = "restaurant_reviews.csv"

In [4]:
lines = sc.textFile(f"file:///{directory}/{filename}")

In [6]:
lines.collect()

['id,item,category,reviews',
 '0,짜장면,중식,125',
 '1,짬뽕,중식,235',
 '2,김밥,분식,323',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [7]:
header = lines.first()

In [8]:
filtered_lines = lines.filter(lambda row: row != header)

In [9]:
filtered_lines.collect()

['0,짜장면,중식,125',
 '1,짬뽕,중식,235',
 '2,김밥,분식,323',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [10]:
def parse(row):
    fields = row.split(",")
    category = fields[2]
    reviews = int(fields[3])
    return (category, reviews)

In [11]:
categoryReviews = filtered_lines.map(parse)

In [12]:
categoryReviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 323),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [13]:
categoryReviewsCount = categoryReviews.mapValues(lambda x: (x,1))

In [14]:
categoryReviewsCount.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (323, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [19]:
reduced = categoryReviewsCount.reduceByKey(lambda x,y : (x[0] + y[0], x[1] + y[1]))

In [20]:
reduced.collect()

[('중식', (360, 2)),
 ('분식', (857, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [21]:
averages = reduced.mapValues(lambda x: x[0] / x[1])
averages.collect()

[('중식', 180.0),
 ('분식', 428.5),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]